### Ingest drivers.json file
#### Nested json

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

### Step 1 - read the JSON file using the spark dataframe reader API

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
                                 
])

In [0]:
driver_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                   StructField("driverRef", StringType(), True),
                                   StructField("number", IntegerType(), True),
                                   StructField("code", StringType(), True),
                                   StructField("name", name_schema),
                                   StructField("dob", DateType(), True),
                                   StructField("nationality", StringType(), True),
                                   StructField("url", StringType(), True)
])

In [0]:
driver_df = spark.read \
.schema(driver_schema) \
.json(f"{raw_folder_path}/{v_file_date}/drivers.json")

### Step 2 - rename the columns and add new columns 
#### 
1. driverId renamed to driver_id
2. driverref renamed to driver_ref
3. ingestion date added
4. name added with concatenation of forename and surname

In [0]:
from pyspark.sql.functions import col, concat, lit

In [0]:
drivers_with_columns_df = add_ingestion_date(driver_df) \
                            .withColumnRenamed("driverId", "driver_id") \
                            .withColumnRenamed("driverRef", "driver_ref") \
                            .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                            .withColumn("data_source", lit(v_data_source)) \
                            .withColumn("file_date", lit(v_file_date))

### Step 3 - Drop the unwanted columns
#### 
1. name.forename
2. name.surname
3. url 

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

#### Step 4 - Write the output to processed container in parquet format

In [0]:
#drivers_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")
drivers_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/drivers"))

In [0]:
%sql
SELECT * 
FROM f1_processed.drivers

In [0]:
dbutils.notebook.exit("Success")